# Moderate Text Using Azure Content Safety API

Start by installing the packages that will be required

In [ ]:
# install the packages
%pip install requests
%pip install azure-ai-contentsafety
%pip install python-dotenv

# Import Packages

Next, you'll import all the packages that are required

In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory, AnalyzeTextOutputType

# Create Content Safety Client

You will create content safety `client` in this step which will be used to pass API requests to Azure servers for moderations.

In [ ]:
# Load your Azure Safety API key and endpoint
load_dotenv()

key = os.environ["CONTENT_SAFETY_KEY"]
endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]

# Create a Content Safety client
client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

# Moderate Text

Implement the `moderate_text` function

In [ ]:
def moderate_text(text):
    # Construct a request
    request = AnalyzeTextOptions(text=text, output_type=AnalyzeTextOutputType.EIGHT_SEVERITY_LEVELS)

    # Analyze text
    try:
        response = client.analyze_text(request)
    except HttpResponseError as e:
        print("Analyze text failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    # Extract results
    categories = {
        TextCategory.HATE: None,
        TextCategory.SELF_HARM: None,
        TextCategory.SEXUAL: None,
        TextCategory.VIOLENCE: None
    }

    for item in response.categories_analysis:
        if item.category in categories:
            categories[item.category] = item

    hate_result = categories[TextCategory.HATE]
    self_harm_result = categories[TextCategory.SELF_HARM]
    sexual_result = categories[TextCategory.SEXUAL]
    violence_result = categories[TextCategory.VIOLENCE]

    # Check for inappropriate content
    violations = []
    if hate_result and hate_result.severity > 2:
        violations.append("hate speech")
    if self_harm_result and self_harm_result.severity > 3:
        violations.append("self-harm references")
    if sexual_result and sexual_result.severity > 0:
        violations.append("sexual references")
    if violence_result and violence_result.severity > 2:
        violations.append("violent references")

    if violations:
        return f"Your content contains {', '.join(violations)} that violate our community guidelines. Please modify your post to adhere to community guidelines."

    # If content is appropriate
    return "Post successful"

In [ ]:
text_to_moderate = "Millions of foolish peeps were wiped out because they were the inferior race. Murdering them continues to be right and honorable because…"
moderation_response = moderate_text(text_to_moderate)
print(moderation_response)